In [1]:
import os
import re
import statistics
from src.najdi_rok import najdi_rok
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
df = pd.read_parquet("data/cnb_vyber.parquet")
df['rok'] = df["008"].apply(lambda x: najdi_rok(x))
df = df[df['rok'].notnull()]
df = df[df['100_7'].notnull()]

In [6]:
stovky = df[['100_7','100_a']].dropna().drop_duplicates()
stovky.sample(10)

,100_7,100_a
001,,
cpk20000960469,jk01091190,"Opletal, Josef Alois,"
cpk20010889501,xx0002804,"Rogers, Rosemary,"
ck9206140,xx0011306,"Rajpoot, Govind S.,"
cpk20011009046,jn20020721219,"Mojay, Gabriel"
nkc20122391343,xx0161652,"Solås, Monica Kristensen,"
nkc20112257160,xx0143194,"Lewis, Charni"
bk194802998,jk01120858,"Staněk, Václav Bohumil,"
nkc20193158562,xx0241427,"Kratochvílová, Tereza,"
cpk19980604257,xx0182181,"Rau, Robert,"


In [3]:
df.groupby("100_7")['rok'].nunique().nlargest(50)

100_7
jk01083016       121
jk01030532       118
jk01051816       118
jn19990008769    116
jn19981002129    111
jk01083209       103
jk01021023       101
jk01072915        99
jk01061444        97
jn19981002230     97
jk01071364        96
jk01110657        88
jk01010468        87
jn19990005499     86
jk01021020        84
jk01101955        84
jk01141424        84
jk01151037        84
jn19990001907     84
jn19990210182     84
jk01040097        82
jk01042920        82
jk01060677        82
jn19990005166     82
jk01090162        81
jk01110710        81
jk01131478        81
jn19990000418     81
jn19990210160     81
jk01030247        80
jk01040361        80
jk01100136        80
jk01130973        79
jk01021137        76
jk01093291        76
jk01152552        76
jn19990006608     76
jk01121895        75
jk01151789        75
jk01062651        74
jk01080472        74
jk01091013        74
jk01140553        74
jn19990000171     74
jn19990210513     74
jk01040882        72
jk01062666        72
jk01092

In [46]:
def jak_dlouho(zkratka):
    rocnik = 2023
    while len(df[(df['100_7'] == zkratka) & (df['rok'] == rocnik)]) > 0:
        rocnik = rocnik - 1
    return rocnik + 1

In [26]:
jak_dlouho('jn19981002230')

2020

In [48]:
nejdelsi_snura = {}
for x in df.groupby("100_7")['rok'].nunique().nlargest(500).index.to_list():
    print(x)
    nejdelsi_snura[x] = jak_dlouho(x)
nejdelsi_snura = pd.Series(nejdelsi_snura).sort_values()

jk01083016
jk01030532
jk01051816
jn19990008769
jn19981002129
jk01083209
jk01021023
jk01072915
jk01061444
jn19981002230
jk01071364
jk01110657
jk01010468
jn19990005499
jk01021020
jk01101955
jk01141424
jk01151037
jn19990001907
jn19990210182
jk01040097
jk01042920
jk01060677
jn19990005166
jk01090162
jk01110710
jk01131478
jn19990000418
jn19990210160
jk01030247
jk01040361
jk01100136
jk01130973
jk01021137
jk01093291
jk01152552
jn19990006608
jk01121895
jk01151789
jk01062651
jk01080472
jk01091013
jk01140553
jn19990000171
jn19990210513
jk01040882
jk01062666
jk01092720
jn19981000482
jn19990001747
jk01090079
jn19990003739
jn19990009450
jk01073121
jn19981002409
jn19990004353
jn19990005488
jk01022199
jk01070421
jk01010356
jn19990007946
jk01021929
jk01033200
jn19981002263
jk01033219
jk01041653
jk01071520
jk01122425
jn19990002740
jn19990002786
jn19990008249
jn19990210244
jn19992019653
jk01052941
jk01072867
jk01011066
jk01011653
jk01110164
jn19990007118
jn19990001842
jk01011993
jk01020045
jk01111743
jk0

In [52]:
df_snura = pd.DataFrame(nejdelsi_snura).reset_index(drop=False).rename(columns={'index': '100_7',0:'odkdy'}).merge(stovky, on='100_7', how='left')

In [54]:
df_snura['delka'] = 2024 - df_snura['odkdy']

In [56]:
df_snura

,100_7,odkdy,100_a,delka
0,jk01092720,1967,"Petiška, Eduard,",57
1,jk01152946,1978,"Žáček, Jiří,",46
2,jn19990008769,1983,"Verne, Jules,",41
3,jk01141586,1987,"Vaňková, Ludmila,",37
4,jk01020709,1987,"Cílek, Roman,",37
5,jk01093291,1990,"Pittnerová, Vlasta,",34
6,jn19990005095,1991,"Lindgren, Astrid,",33
7,jn19981000482,1991,"Christie, Agatha,",33
8,jn19990001513,1991,"Courths-Mahler, Hedwig,",33
9,jn19981002129,1991,"Shakespeare, William,",33
